In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import tifffile as tiff

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

import hyper_img as hyper

In [ ]:
IMG_PATH = '/home/igor/projects/Seed/data/golozernost'
CALIBR_WHITE_PATH = '/home/igor/projects/Seed/data/golozernost/session_001_004_snapshot_cube.tiff'
CALIBR_BLACK_PATH = '/home/igor/projects/Seed/data/golozernost/session_001_005_snapshot_cube.tiff'

In [ ]:
class GolozernostHyperImg(hyper.HyperImg):
    
    def _get_tiff(self) -> np.ndarray:
        img = tiff.imread(self.path)
        bl_img = tiff.imread(CALIBR_BLACK_PATH)
        new_img = np.where(bl_img > img, 0, img - bl_img)
        wh_img = tiff.imread(CALIBR_WHITE_PATH)
        return new_img /(wh_img - bl_img)

    
    def _get_target_varible(self) -> str:
        if not set(IMG_PATH.split('/')).issubset(set(self.path.split('.')[0].split('/'))):
            raise NameError('Error in path')
        
        if 'голозерные' in self.path:
            return 'голозерные'
        
        if 'пленчатые' in self.path:
            return 'пленчатые'
        
        return 'калибровка'

In [ ]:
img_lst: list[GolozernostHyperImg] = hyper.get_list_hyper_img(path=IMG_PATH, 
                                                       class_name=GolozernostHyperImg,
                                                       filter=lambda x: x != 'калибровка', 
                                                       threshold_value=25, 
                                                       target_varible_name= 'Голозерность/Пленчатость')

In [ ]:
img_lst[0]

In [ ]:
len(img_lst)

In [ ]:
plt.figure(figsize=(7,5))
plt.grid()
sns.lineplot(data=hyper.get_df_graphics_medians_wavelenght(img_lst), 
             x='Wavelength', y='Median', hue='Голозерность/Пленчатость')

In [ ]:
plt.figure(figsize=(15,7))
sns.scatterplot(data=pd.DataFrame(hyper.get_df_2_pca(img_lst), 
                                  columns=['1', '2', 'Голозерность/Пленчатость']), x='1', y='2', hue='Голозерность/Пленчатость')

In [ ]:
pipe = Pipeline([('scaler', StandardScaler()), ('pca', PCA(n_components=5))])

In [ ]:
df = hyper.get_df_medians(img_lst)
df.head()

In [ ]:
X = df.drop(['Голозерность/Пленчатость'], axis = 1)
y = df[['Голозерность/Пленчатость']]
X = pipe.fit_transform(X)
pipe['pca'].explained_variance_ratio_